In [6]:
# include our Python packages
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.autograd import Function
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

from torch.utils.data import Dataset, DataLoader


In [7]:

class MLP(nn.Module):
    def __init__(self, D_in, H, D_out) -> None:
        super(MLP).__init__()
        self.fc1 = nn.Linear(D_in, H)
        self.fc2 = nn.Linear(H, D_out)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x

# Build dataloader

In [65]:
def to_bool(str):
    if str.lower() == 'yes':
        return float(1.00)
    elif str.lower() == 'no':
        return float(0.00)

In [115]:
import torch
import torch.nn.functional as F

# Create a sample 2D tensor
tensor_2d = torch.tensor([[1.0, 2.0, 3.0],
                          [4.0, 5.0, 6.0]])

print(tensor_2d[1,1:])
# Specify the dimension along which to normalize
dimension_to_normalize = 0 # 0: column

# Apply normalization
normalized_tensor = F.normalize(tensor_2d, p=2, dim=dimension_to_normalize)

# Display the original and normalized tensors
print("Original Tensor:")
print(tensor_2d)

print("\nNormalized Tensor:")
print(normalized_tensor)


tensor([5., 6.])
Original Tensor:
tensor([[1., 2., 3.],
        [4., 5., 6.]])

Normalized Tensor:
tensor([[0.2425, 0.3714, 0.4472],
        [0.9701, 0.9285, 0.8944]])


In [120]:
from typing import Any


class StudentDataset(Dataset):
    def __init__(self, csv_file, transform=None):

        self.df = pd.read_csv(csv_file)
        self.df.iloc[:,2] = self.df.iloc[:,2].apply(lambda x: to_bool(x)) # convert yes-no's to 1-0's
        self.df = self.df.astype('int') # convert to int type

        self.transform = transform
        
    def __getitem__(self, index):
        # convert to tensors
        X = torch.tensor(self.df.iloc[index,:-1], dtype=torch.float32) # get features
        y = torch.tensor(self.df.iloc[index,-1], dtype=torch.float32)

        sample = X, y

        if self.transform:
            sample = self.transform(sample)

        return sample
    
    def __len__(self):
        return len(self.data)


class normalise:
    def __call__(self, sample):
        X, y = sample
        X1 = F.normalize(input=X[:2], p=2, dim=0)
        X2 = X[2]
        X3 = F.normalize(input=X[3:], p=2, dim=0)

        X_new = torch.cat((X1,X2,X3))

        return X_new



In [121]:
PATH = "C:/Users/Zayan/Documents/datasets/kaggle_student_performance"
student_data = StudentDataset(csv_file=f"{PATH}/Student_Performance.csv", transform=normalise())

In [122]:
student_data[3]

C:\Users\Zayan\AppData\Local\Temp\ipykernel_10184\857746957.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  X = torch.tensor(self.df.iloc[index,:-1], dtype=torch.float32) # get features


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [78]:
df = pd.read_csv(f"{PATH}/Student_Performance.csv")

In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Hours Studied                     10000 non-null  int64  
 1   Previous Scores                   10000 non-null  int64  
 2   Extracurricular Activities        10000 non-null  object 
 3   Sleep Hours                       10000 non-null  int64  
 4   Sample Question Papers Practiced  10000 non-null  int64  
 5   Performance Index                 10000 non-null  float64
dtypes: float64(1), int64(4), object(1)
memory usage: 468.9+ KB


In [82]:
df.iloc[:,2] = df.iloc[:,2].apply(lambda x: to_bool(x))

In [83]:
df

,Hours Studied,Previous Scores,Extracurricular Activities,Sleep Hours,Sample Question Papers Practiced,Performance Index
0,7,99,1.0,9,1,91.0
1,4,82,0.0,4,2,65.0
2,8,51,1.0,7,2,45.0
3,5,52,1.0,5,2,36.0
4,7,75,0.0,8,5,66.0
...,...,...,...,...,...,...
9995,1,49,1.0,4,2,23.0
9996,7,64,1.0,8,5,58.0
9997,6,83,1.0,8,5,74.0
9998,9,97,1.0,7,0,95.0


In [88]:
df = df.astype('int')

In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column                            Non-Null Count  Dtype
---  ------                            --------------  -----
 0   Hours Studied                     10000 non-null  int32
 1   Previous Scores                   10000 non-null  int32
 2   Extracurricular Activities        10000 non-null  int32
 3   Sleep Hours                       10000 non-null  int32
 4   Sample Question Papers Practiced  10000 non-null  int32
 5   Performance Index                 10000 non-null  int32
dtypes: int32(6)
memory usage: 234.5 KB
